In [1]:
# 计算性能 
''' 
在深度学习中，数据集和模型通常都很⼤，导致计算量也会很⼤。因此，计算的性能⾮常重要。本章将集中
讨论影响计算性能的主要因素：命令式编程、符号编程、异步计算、⾃动并⾏和多GPU计算。通过学习本章，
对于前⼏章中实现的那些模型，你可以进⼀步提⾼它们的计算性能。例如，我们可以在不影响准确性的前提
下，⼤⼤减少训练时间。
'''
# 编译器和解释器
''' 
⽬前为⽌，本书主要关注的是命令式编程（imperative programming）。命令式编程使⽤诸如print、“+”
和if之类的语句来更改程序的状态。考虑下⾯这段简单的命令式程序：

Python是⼀种解释型语⾔（interpreted language）。因此，当对上⾯的fancy_func函数求值时，它按顺序
执⾏函数体的操作。也就是说，它将通过对e = add(a, b)求值，并将结果存储为变量e，从⽽更改程序的
状态。接下来的两个语句f = add(c, d)和g = add(e, f)也将执⾏类似地操作，即执⾏加法计算并将
结果存储为变量。

尽管命令式编程很⽅便，但可能效率不⾼。⼀⽅⾯原因，Python会单独执⾏这三个函数的调⽤，⽽没有考
虑add函数在fancy_func中被重复调⽤。如果在⼀个GPU（甚⾄多个GPU）上执⾏这些命令，那么Python解
释器产⽣的开销可能会⾮常⼤。此外，它需要保存e和f的变量值，直到fancy_func中的所有语句都执⾏完
毕。这是因为程序不知道在执⾏语句e = add(a, b)和f = add(c, d)之后，其他部分是否会使⽤变
量e和f
'''
def add(a, b):
    return a + b
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))

10


In [6]:
# 符号试编程
''' 
考虑另⼀种选择符号式编程（symbolic programming），即代码通常只在完全定义了过程之后才执⾏计算。
这个策略被多个深度学习框架使⽤，包括Theano和TensorFlow（后者已经获得了命令式编程的扩展）。⼀般
包括以下步骤：
1. 定义计算流程。
2. 将流程编译成可执⾏的程序。
3. 给定输⼊，调⽤编译好的程序执⾏。
这将允许进⾏⼤量的优化。⾸先，在⼤多数情况下，我们可以跳过Python解释器。从⽽消除因为多个更快
的GPU与单个CPU上的单个Python线程搭配使⽤时产⽣的性能瓶颈。其次，编译器可以将上述代码优化和重
写为print((1 + 2) + (3 + 4))甚⾄print(10)。因为编译器在将其转换为机器指令之前可以看到完
整的代码，所以这种优化是可以实现的。例如，只要某个变量不再需要，编译器就可以释放内存（或者从不
分配内存），或者将代码转换为⼀个完全等价的⽚段。下⾯，我们将通过模拟命令式编程来进⼀步了解符号式
编程的概念
'''
def add_():
    return '''
def add(a, b):
    return a + b
'''
def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''
def evoke_(): # evoke 产生,引起
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'
prog = evoke_() # str类型
print(prog)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))


In [5]:
# compile方法 编译代码为代码对象，可以提高效率：
''' 
compile(source, filename, mode)

source为代码语句的字符串；如果是多行语句，则每一行的结尾必须有换行符\n

filename为包含代码的文件

mode为编码方式，可以为’exec’（用于语句序列的执行），可以为’eval’（用于表达式求值），
可以为’single’（用于单个交互语句)

'''
y = compile(prog, '', 'exec') # code类型


In [13]:
''' 
compile(source, filename, mode, flags=0, dont_inherit=False, optimize=-1)

source：字符串或AST对象，表示需要进行编译的python代码
filename：指定需要编译的代码文件，如果不是文件读取代码则传递一些可辨认的值。
mode：用于标识必须当做那类代表来编译；如果source是由一个代码语句序列组成，则指定mode=‘exec’，
如果source由单个表达式组成，则指定mode=‘eval’；如果source是由一个单独的交互式语句组成，
则指定modo=‘single’。必须要制定，不然肯定会报错。=
---------------------------------------------------------------------------------
exec(object[, globals[, locals]])

object：必选参数，表示需要被指定的 Python 代码。它必须是 字符串 或 code 对象
如果 object 是一个字符串，该字符串会先被解析为一组 Python 语句，然后再执行（除非发生语法错误）
如果 object 是一个 code 对象，那么它只是被简单的执行
globals：可选参数，表示全局命名空间（存放全局变量），如果被提供，则必须是一个字典对象
locals：可选参数，表示当前局部命名空间（存放局部变量），如果被提供，可以是任何映射对象
如果该参数被忽略，那么它将会取与 globals 相同的值
返回值：
exec函数的返回值永远为None
----------------------------------------------------------------------
eval()函数和exec()函数的区别：

eval()函数只能计算单个表达式的值，而exec()函数可以动态运行代码段。

eval()函数可以有返回值，而exec()函数返回值永远为None
'''
# Code对象表示字节编译的可执行Python代码或字节码
exec(y) # exec() 动态执行Python代码,不同于eval()只能计算一个表达式的值
exec(prog) # exec(object) 必须为字符串或code对象


10


In [ ]:
''' 
命令式（解释型）编程和符号式编程的区别如下：
• 命令式编程更容易使⽤。在Python中，命令式编程的⼤部分代码都是简单易懂的。命令式编程也更容
易调试，这是因为⽆论是获取和打印所有的中间变量值，或者使⽤Python的内置调试⼯具都更加简单。
• 符号式编程运⾏效率更⾼，更易于移植。符号式编程更容易在编译期间优化代码，同时还能够将程序移
植到与Python⽆关的格式中，从⽽允许程序在⾮Python环境中运⾏，避免了任何潜在的与Python解释
器相关的性能问题
'''

In [14]:
# 混合式编程
''' 
历史上，⼤部分深度学习框架都在命令式编程与符号式编程之间进⾏选择。例如，Theano、TensorFlow（灵
感来⾃前者）、Keras和CNTK采⽤了符号式编程。相反地，Chainer和PyTorch采取了命令式编程。在后来的
版本更新中，TensorFlow2.0和Keras增加了命令式编程。
如上所述，PyTorch是基于命令式编程并且使⽤动态计算图。为了能够利⽤符号式编程的可移植性和效率，开
发⼈员思考能否将这两种编程模型的优点结合起来，于是就产⽣了torchscript。torchscript允许⽤⼾使⽤纯
命令式编程进⾏开发和调试，同时能够将⼤多数程序转换为符号式程序，以便在需要产品级计算性能和部署
时使⽤。
'''

' \n历史上，⼤部分深度学习框架都在命令式编程与符号式编程之间进⾏选择。例如，Theano、TensorFlow（灵\n感来⾃前者）、Keras和CNTK采⽤了符号式编程。相反地，Chainer和PyTorch采取了命令式编程。在后来的\n版本更新中，TensorFlow2.0和Keras增加了命令式编程。\n如上所述，PyTorch是基于命令式编程并且使⽤动态计算图。为了能够利⽤符号式编程的可移植性和效率，开\n发⼈员思考能否将这两种编程模型的优点结合起来，于是就产⽣了torchscript。torchscript允许⽤⼾使⽤纯\n命令式编程进⾏开发和调试，同时能够将⼤多数程序转换为符号式程序，以便在需要产品级计算性能和部署\n时使⽤。\n'

In [15]:
# Sequential的混合式编程
''' 
要了解混合式编程的⼯作原理，最简单的⽅法是考虑具有多层的深层⽹络。按照惯例，Python解释器需要执
⾏所有层的代码来⽣成⼀条指令，然后将该指令转发到CPU或GPU。对于单个的（快速的）计算设备，这不会导
致任何重⼤问题。另⼀⽅⾯，如果我们使⽤先进的8-GPU服务器，⽐如AWS P3dn.24xlarge实例，Python将很难
让所有的GPU都保持忙碌。在这⾥，瓶颈是单线程的Python解释器。让我们看看如何通过将Sequential替
换为HybridSequential来解决代码中这个瓶颈。⾸先，我们定义⼀个简单的多层感知机
'''
import torch
from torch import nn
from d2l import torch as d2l
# ⽣产⽹络的⼯⼚模式
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
                        nn.ReLU(),
                        nn.Linear(256, 128),
                        nn.ReLU(),
                        nn.Linear(128, 2))
    return net
x = torch.randn(size=(1, 512)) # 正态分布
net = get_net() # 实例化
net(x)

tensor([[0.0269, 0.1030]], grad_fn=<AddmmBackward0>)

In [16]:
''' 
通过使⽤torch.jit.script函数来转换模型，我们就有能⼒编译和优化多层感知机中的计算，⽽模型的
计算结果保持不变。

术语
Tochscript:狭义概念导出图形的表示/格式;广义概念为导出模型的方法;
(Torch)Scriptable:可以用torch.jit.script导出模型
Traceable:可以用torch.jit.trace导出模型

什么时候用torch.jit.trace（结论：首选）
torch.jit.trace一种导出方法;它运行具有某些张量输入的模型，并“跟踪/记录”所有执行到图形中的操作。
在模型内部的数据类型只有张量，且没有for if while等控制流，选择torch.jit.trace
支持python的预处理和动态行为；
torch.jit.trace编译function并返回一个可执行文件，该可执行文件将使用即时编译进行优化。
大项目优先选择torch.jit.trace,特别是是图像检测和分割的算法；
优点:
不会损害代码质量；
它的主要限制可以通过与torch.jit.script混合来解决

什么时候用torch.jit.script（结论：必要时）
定义:一种模型导出方法,其实编译python的模型源码,得到可执行的图;
在模型内部的数据类型只有张量，且没有for if while等控制流，也可以选择torch.jit.script
不支持python的预处理和动态行为；
必须做一下类型标注；

torch.jit.script在编译function或 nn.Module 脚本将检查源代码，
使用 TorchScript 编译器将其编译为 TorchScript 代码,即命令式编程转变为符号式编程
'''
net = torch.jit.script(net) # 模型转换 pytorch 导出模型有两种方式 torch.jit.script，torch.jit.trace
net(x)                      # 如果网络中用到了 索引，if 条件判断等，很有可能会失败

tensor([[0.0269, 0.1030]], grad_fn=<AddmmBackward0>)

In [17]:
''' 
我们编写与之前相同的代码，再使⽤torch.jit.script简单地转换模型，当完成这些任务后，⽹络就将
得到优化（我们将在下⾯对性能进⾏基准测试)
'''
# 通过混合式编程加速
''' 
为了证明通过编译获得了性能改进，我们⽐较了混合编程前后执⾏net(x)所需的时间。让我们先定义⼀个
度量时间的类，它在本章中在衡量（和改进）模型性能时将⾮常有⽤
'''
#@save
class Benchmark: # 基准
    """⽤于测量运⾏时间"""
    def __init__(self, description='Done'):
        self.description = description # 描述,形容;种类,类型
    def __enter__(self):
        self.timer = d2l.Timer()
        return self
    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

In [18]:
# 现在我们可以调⽤⽹络两次，⼀次使⽤torchscript，⼀次不使⽤torchscript
''' 
torchscript允许用户使⽤纯命令式编程进⾏开发和调试，
同时能够将⼤多数程序转换为符号式程序，以便在需要产品级计算性能和部署时使⽤
'''
net = get_net()
with Benchmark('⽆torchscript'):
    for i in range(1000): net(x)
    
net = torch.jit.script(net) # 将模型转换为符号式程序 符号式程序运行效率更高,更易于移植
with Benchmark('有torchscript'):
    for i in range(1000): net(x)

⽆torchscript: 0.0523 sec
有torchscript: 0.0800 sec


In [19]:
''' 
如以上结果所⽰，在nn.Sequential的实例被函数torch.jit.script脚本化后，通过使⽤符号式编程提
⾼了计算性能。
'''
# 序列化
''' 
编译模型的好处之⼀是我们可以将模型及其参数序列化（保存）到磁盘。这允许这些训练好的模型部署到其
他设备上，并且还能⽅便地使⽤其他前端编程语⾔。同时，通常编译模型的代码执⾏速度也⽐命令式编程更
快。让我们看看save的实际功能
'''
net.save('my_mlp') # 保存参数
!ls -lh my_mlp*

-rw-r--r-- 1 ubuntu ubuntu 652K Oct  6 21:19 my_mlp


In [ ]:
''' 
⼩结
• 命令式编程使得新模型的设计变得容易，因为可以依据控制流编写代码，并拥有相对成熟的Python软件⽣态
• 符号式编程要求我们先定义并且编译程序，然后再执⾏程序，其好处是提⾼了计算性能
'''